In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09e_tflite.ipynb"
_nb_title = "Edge ML with TensorFlow Lite"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_title, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Edge ML with TensorFlow Lite&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F09_deploying%2F09e_tflite.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F09_deploying%2F09e_tflite.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09e_tflite.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09e_tflite.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/09_deploying/09e_tflite.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# TensorFlowLiteを使用したEdgeML  

このノートブックでは、保存したモデルをTensorFlowLiteモデルに変換します  
エッジデバイスで実行できるようにします。  

エッジ推論を行うには、カメラからの生の画像データを処理する必要があります  
(画像のバッチではなく)単一の画像を処理します。

In [2]:
import tensorflow as tf
import os, shutil

MODEL_LOCATION='export/flowers_model3'  # will be created
# load from checkpoint and export a model that has desired signature
CHECK_POINT_DIR='gs://practical-ml-vision-book/flowers_5_trained/chkpts'
model = tf.keras.models.load_model(CHECK_POINT_DIR)

In [3]:
IMG_HEIGHT = 345
IMG_WIDTH = 345
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()
    
# a single image of any size
@tf.function(input_signature=[tf.TensorSpec([None, None, 3], dtype=tf.float32)])
def predict_flower_type(img):
    img = tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)
    batch_pred = model(tf.expand_dims(img, axis=0))
    top_prob = tf.math.reduce_max(batch_pred, axis=[1])
    pred_label_index = tf.math.argmax(batch_pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES), pred_label_index)
    return {
        'probability': tf.squeeze(top_prob, axis=0),
        'flower_type': tf.squeeze(pred_label, axis=0)
    }

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')


model.save(MODEL_LOCATION,
          signatures={
              'serving_default': predict_flower_type
          })

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: export/flowers_model3/assets


## TFLiteに変換  

変換にはしばらく時間がかかります

In [4]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_LOCATION)
tflite_model = converter.convert()

In [7]:
with open('export/model.tflite', 'wb') as ofp:
    ofp.write(tflite_model)

In [8]:
!ls -lh export/model.tflite

-rw-r--r-- 1 jupyter jupyter 8.8M Jan 26 05:31 export/model.tflite


## ライセンス  
Copyright 2020 Google Inc. Apache Licenseバージョン2.0(「ライセンス」)に基づいてライセンス供与されます。ライセンスに準拠する場合を除き、このファイルを使用することはできません。ライセンスのコピーはhttp://www.apache.org/licenses/LICENSE-2.0で入手できます。適用法で要求されているか、書面で合意されていない限り、ライセンスに基づいて配布されるソフトウェアは「現状有姿」で配布されます。明示または黙示を問わず、いかなる種類の保証または条件もありません。ライセンスに基づく許可と制限を規定する特定の言語については、ライセンスを参照してください。